In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing as pre
import glob
from PIL import Image as im
import matplotlib.pyplot as plt
from pylab import *
from Lib import random

np.random.seed(0)

%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.__version__

'1.14.0'

In [3]:
class FLAGS():
    pass
FLAGS.model_dir='\\Users\\shiely\\Projects\\main_outrig\\ML4LITHO'
#FLAGS.model_dir = '\\Users\\shiely\\Projects\\main_outrig\\auto-encoder-04\\context-off-49'
FLAGS.data_dir = '\\Users\\shiely\\Projects\\main_outrig'
FLAGS.model_name = 'context_off_49'
FLAGS.batch_size = 128
FLAGS.learning_rate = 0.0001
FLAGS.l2_reg = 0.05
FLAGS.noise_level = 0.00
FLAGS.dropout_rate = 0.5
FLAGS.resume_training = False
FLAGS.max_steps = 1000000
FLAGS.eval_steps = 100# Customize for 
#FLAGS.logdir = '\\Users\\shiely\\Projects\\main_outrig\\auto-encoder-04\\context-off-49'
FLAGS.logdir = '\\Users\\shiely\\Projects\\main_outrig\\logs\\ML4LITHO'
FLAGS.projectordir = FLAGS.logdir +'projector\\'
FLAGS.dec_in_channels = 1
FLAGS.n_latent = 64
FLAGS.image_dim = 48
FLAGS.beta=0.05
FLAGS.percent_of_train=90
FLAGS.sample_size=800
FLAGS.training_epochs=10000
FLAGS.context_on=False
FLAGS.shuffle_data=True
LOGDIR=FLAGS.logdir

scopes=['NetLayer/LogitsLayer/Logits:0','NetLayer/MetricsLayer/labels:0']
summary_writer = tf.compat.v1.summary.FileWriter(FLAGS.logdir)

In [4]:
from design_target_to_mask_problem import mask_design_and_context_data_generation
data_sets= mask_design_and_context_data_generation(FLAGS)

loaded masks, designs and contects of shapes (800, 48, 48) (800, 48, 48) (800, 48, 48) and 800 elements.
everybodys shuffling : shuf? True perc 90 start_test_index 640 data shape 800
shuffled shape (800, 48, 48)


In [5]:
# first make it work for VAE (tues)
# then make it work for linear, perceptrion, mlp, cnn (weds)
# then make it work for gan (thurs)
# then put on github (fri)
# then write slides (fri-sat-sun)



In [6]:
def generate_input_fn(data,
                      mode=tf.estimator.ModeKeys.EVAL,
                      batch_size=1): 
    def input_fn():
        if mode == tf.estimator.ModeKeys.EVAL:
            masks   = data.test.masks
            designs = data.test.designs
            contexts= data.test.contexts
            print('loading training input : masks {} designs {} contexts {}'.
                  format(masks.shape,designs.shape,contexts.shape))
            num_epochs = 1
        elif mode   == tf.estimator.ModeKeys.TRAIN:
            masks   = data.train.masks
            designs = data.train.designs
            contexts= data.train.contexts
            print('loadingeval input : masks {} designs {} contexts {}'.
                  format(masks.shape,designs.shape,contexts.shape))
            num_epochs=None
        else:
            assert(False)

        is_shuffle = (mode == tf.estimator.ModeKeys.TRAIN)

        return tf.compat.v1.estimator.inputs.numpy_input_fn(
          batch_size=batch_size,
          shuffle=is_shuffle,
          x={'masks': masks,'designs': designs, 'contexts':contexts},
          y=None,
          num_epochs=num_epochs)

    return input_fn()

In [7]:
def serving_input_fn():
    input_masks= tf.placeholder(shape=[None, FLAGS.image_dim*FLAGS.image_dim], dtype=tf.float32, name='masks')
    input_designs = tf.placeholder(shape=[None, FLAGS.image_dim*FLAGS.image_dim], dtype=tf.float32, name='designs')
    input_contexts= tf.placeholder(shape=[None, FLAGS.image_dim*FLAGS.image_dim], dtype=tf.float32, name='contexts')

    features ={'masks' : input_masks, 'designs' : input_designs, 'contexts': input_contexts}
    receiver_tensors = {'masks' : input_masks, 'designs' : input_designs, 'contexts' : input_contexts}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

In [8]:
from variational_auto_encoder_model import get_bayes_estimator

In [ ]:
import os as os
run_config = tf.estimator.RunConfig(
    model_dir=os.path.join(FLAGS.model_dir, FLAGS.model_name),
    save_checkpoints_steps=1000,
    keep_checkpoint_max=2,
    tf_random_seed=19851211
)

#from tensorboard.plugins.hparams import api as hp

hparams = {
    "batch_size" : FLAGS.batch_size,
    "encoder_hidden_units" : FLAGS.n_latent,
    "learning_rate"  :FLAGS.learning_rate,
    "l2_reg" : FLAGS.l2_reg,
    "noise_level" : FLAGS.noise_level,
    "dropout_rate" : FLAGS.dropout_rate,
    "image_dim": FLAGS.image_dim,
    "n_latent":FLAGS.n_latent,
    "context_on":FLAGS.context_on,
    "beta":FLAGS.beta
}

estimator = get_bayes_estimator(run_config, hparams)

train_spec = tf.estimator.TrainSpec(
    input_fn=generate_input_fn(data_sets,
                             mode=tf.estimator.ModeKeys.TRAIN,
                             batch_size=FLAGS.batch_size),
    max_steps=FLAGS.max_steps, hooks=None)
train_input_fn=generate_input_fn(data_sets,
                             mode=tf.estimator.ModeKeys.TRAIN,
                             batch_size=FLAGS.batch_size)

exporter = tf.estimator.LatestExporter(
      name='Servo',
      serving_input_receiver_fn=serving_input_fn,
      exports_to_keep=5)

eval_spec = tf.estimator.EvalSpec(
    input_fn=generate_input_fn(data_sets),
    steps=FLAGS.eval_steps, exporters=exporter, throttle_secs=300)
import shutil
if not FLAGS.resume_training:
    print('Removing previous artifacts...')
    shutil.rmtree(os.path.join(FLAGS.model_dir, FLAGS.model_name), ignore_errors=True)

#estimator.train(train_input_fn)
tf.compat.v1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': '\\Users\\shiely\\Projects\\main_outrig\\ML4LITHO\\context_off_49', '_tf_random_seed': 19851211, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000024E80603D08>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
loadingeval input : masks (720, 48, 48) designs (720, 48, 48) contexts (720, 

In [ ]:
estimator.export_saved_model('saved_model',serving_input_fn)

In [ ]:
data_sets.test.masks.shape

In [ ]:
export_dir =os.path.join(FLAGS.model_dir, FLAGS.model_name, 'export\\Servo\\')
saved_model_dir = '\\Users\\shiely\\saved_model\\1572404268'

predictor_fn = tf.contrib.predictor.from_saved_model(
  export_dir = saved_model_dir,
  signature_def_key='predict')

output = predictor_fn({'masks': np.reshape(data_sets.train.masks[:100],[-1,FLAGS.image_dim*FLAGS.image_dim]),
                      'designs': np.reshape(data_sets.train.designs[:100],[-1,FLAGS.image_dim*FLAGS.image_dim]),
                      'contexts': np.reshape(data_sets.train.contexts[:100],[-1,FLAGS.image_dim*FLAGS.image_dim])})

In [ ]:
x = 19

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,4))
ax1.set_title('original mask')
ax1.imshow(data_sets.train.masks[x].reshape(FLAGS.image_dim,FLAGS.image_dim))

ax2.set_title('reconstructed mask')
ax2.imshow(output['mask_reconstruction'][x].reshape(FLAGS.image_dim,FLAGS.image_dim))
plt.show()

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,4))

ax1.set_title('original design')
ax1.imshow(data_sets.train.designs[x].reshape(FLAGS.image_dim,FLAGS.image_dim))

ax2.set_title('reconstructed design')
ax2.imshow(output['design_reconstruction'][x].reshape(FLAGS.image_dim,FLAGS.image_dim))

plt.show()

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,4))

ax1.set_title('original context')
ax1.imshow(data_sets.train.contexts[x].reshape(FLAGS.image_dim,FLAGS.image_dim))

ax2.set_title('reconstructed context')
ax2.imshow(output['context_reconstruction'][x].reshape(FLAGS.image_dim,FLAGS.image_dim))

plt.show()